In [15]:
from sklearn.ensemble import RandomForestRegressor
from utils import get_data_per_anchor
from sklearn.model_selection import GroupShuffleSplit
import pandas as pd
import numpy as np
from micromlgen import port
import emlearn
from utils import get_data_per_anchor
from sklearn.model_selection import GroupKFold

In [4]:
df = pd.read_pickle("mid_concrete.pkl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
features = ['power', 'pdda_input_real_1', 'pdda_input_real_2', 'pdda_input_imag_2', 'pdda_input_real_3', 'pdda_input_imag_3', 'pdda_input_real_4', 'pdda_input_imag_4',
           'pdda_input_real_5', 'pdda_input_imag_5']
# theta does not change, make predictions for phi only, azimuth angles 
# predict only for phi, 
targets = ['true_phi']
location = ['x_tag', 'y_tag']
pdda_angle = ['pdda_angle']

params = {
    "test_size":0.33,
    "random_state":42,
    "splits":1,
    
}

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

anchor_df = get_data_per_anchor(df,1.0)
X = anchor_df[features].values
y = anchor_df[targets].values

group_kfold = GroupKFold(n_splits=2)
# group_kfold.get_n_splits(X, y, anchor_df.point)

for train_index, test_index in group_kfold.split(X, y, groups=anchor_df.point):
    train_index = shuffle(train_index)
    test_index = shuffle(test_index)
    print("TRAIN:", train_index, "TEST:", test_index)
    


TRAIN: [5570 5580 1460 ... 7308 5327 4405] TEST: [1047 2550 3443 ... 4798  953 6034]
TRAIN: [6960 6986 5942 ...  720 6180 3283] TEST: [1733 6940  747 ... 6207 5179 7123]


In [18]:
for anchor in [1.0,2.0,3.0,4.0]:
    anchor_df = get_data_per_anchor(df,anchor)
    X = anchor_df[features].values
    y = anchor_df[targets].values
    gss = GroupShuffleSplit(n_splits=params["splits"], test_size=params["test_size"], random_state=params["random_state"])
    fold = 1
    for train_idx, test_idx in gss.split(X, y, groups=anchor_df.point):
        print('a')
        train = anchor_df.iloc[train_idx]
        test = anchor_df.iloc[test_idx]
        assert round(test.shape[0]/(train.shape[0]+test.shape[0]), 2) == params["test_size"]
        assert len(set(train.point) & set(test.point)) == 0
        X_train = train[features].values
        y_train = train[targets].values
        X_test = test[features].values
        y_test = test[targets].values

        regr = RandomForestRegressor(random_state=params['random_state'],n_estimators=3)
        regr.fit(X_train,y_train.ravel())
    break

a


In [19]:
X_test

array([[ 0.00313989,  0.11830609,  0.11470547, ..., -0.06965832,
         0.00028239, -0.0272559 ],
       [-0.0446148 ,  0.06654441,  0.07474081, ..., -0.01227114,
        -0.04082246,  0.06268334],
       [-0.02214201,  0.09091801,  0.09127097, ..., -0.03846562,
        -0.02715925,  0.02049112],
       ...,
       [ 0.05229913,  0.12651584,  0.1174032 , ...,  0.14363903,
        -0.17442854,  0.03641184],
       [ 0.02280359,  0.05932768,  0.06124395, ...,  0.10409687,
        -0.15449992,  0.01894717],
       [ 0.06072643,  0.13848052,  0.0147295 , ...,  0.1500174 ,
        -0.1242835 ,  0.03749986]])

In [9]:
y_test

NameError: name 'niakslmasy_test' is not defined

In [7]:
regr

RandomForestRegressor(n_estimators=3, random_state=42)

## micromlgen

In [9]:
with open('RandomForestRegressor.h', 'w') as file:
    file.write(port(regr))

## micromlgen -> creates a c++ header, not a c header

## emlearn only supports classifiers

In [9]:
# cmodel = emlearn.convert(regr, method='inline')
# cmodel.save(file='sonar.h')

In [10]:
rf = regr

In [2]:
for i in range(len(rf.estimators_)):
    tree = rf.estimators_[i].tree_
    df = pd.DataFrame({"Column":tree.feature, "Threshold":tree.threshold,
                        "LeftChild":tree.children_left, "RightChild":tree.children_right,
                        "Value":tree.value.reshape(-1)})
    df = df[["Column","Threshold", "LeftChild", "RightChild", "Value"]]
    df_string = f"{len(df)}\n{df.to_csv(header=False, index=False)}"
    with open(f"tree{i}.csv", "w") as f:
        f.write(df_string)

NameError: name 'rf' is not defined

In [16]:
df[features].head()

,power,pdda_input_real_1,pdda_input_real_2,pdda_input_imag_2,pdda_input_real_3,pdda_input_imag_3,pdda_input_real_4,pdda_input_imag_4,pdda_input_real_5,pdda_input_imag_5
0,-0.251084,0.011985,-0.004184,0.011310,-0.009187,-0.007796,0.011729,0.002421,0.011187,0.004386
1,-0.251084,0.012268,-0.004163,0.011728,-0.009701,-0.007943,0.012262,0.002381,0.011548,0.004437
2,-0.252488,0.011138,-0.003596,0.010398,-0.008614,-0.006782,0.010879,0.001815,0.010350,0.003906
3,-0.071301,0.027557,0.037822,-0.007849,0.049091,-0.012876,-0.003899,0.023019,-0.075814,-0.030990
4,-0.210352,0.008001,0.012124,-0.000136,0.013350,-0.001056,-0.003787,-0.016711,-0.018601,-0.007845
